In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input, Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import metrics
from tensorflow.keras.regularizers import l2

import numpy as np
from sklearn.metrics import classification_report
import time
import os
# import boto3
# import sagemaker
# from sagemaker import get_execution_role
from PIL import Image
import math
import keras.backend as K
import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import scipy
import numpy

print("SciPy 버전:", scipy.__version__)
print("NumPy 버전:", numpy.__version__)

SciPy 버전: 1.11.4
NumPy 버전: 1.25.2


## 파일 경로 확인 작업

In [ ]:
import os

directory = '/content/drive/MyDrive/Inception_model_data/101_105'
contents = os.listdir(directory)
num_of_dirs = len([name for name in contents if os.path.isdir(os.path.join(directory, name))])

print("Contents of the directory:")
for item in contents:
    print(item)

print(f"\nNumber of directories: {num_of_dirs}")

Contents of the directory:
감자그라탕
감자구이
감자칩
가츠동
가지구이
가지
감
갈비구이
감자샐러드
감자스프
고등어
고구마
고등어구이
감자튀김
건자두
경단
건포도
게장
거봉포도
게맛살
고르곤졸라피자
고추장아찌
군고구마
구아바
고르곤졸라
군밤
곶감
과일채소샐러드
국수
고추
귤
까망베르치즈
김밥
김마끼
김치찌개
김치볶음밥
그라탕
김
껌
까르보나라
김치국
꼬막
꿀
단호박
누텔라
단감
녹색피망
달걀볶음밥
나시고랭
다시마
다크초콜릿
냉면
달걀
달걀후라이
달걀샌드위치
달걀비빔밥

Number of directories: 56


In [ ]:
from PIL import Image
import os

# 디렉토리 경로 정의
directory_path = '/content/drive/MyDrive/Inception_model_data/101_105'

# 디렉토리의 모든 파일(음식) 리스트 저장
file_names = os.listdir(directory_path)

# 디렉토리로부터 이미지 로드 [근데 이미지 있는 파일까지 안 내려]
images = []
for file_name in file_names:
    if file_name.endswith('.jpg'):
        image_path = os.path.join(directory_path, file_name)
        image = Image.open(image_path)
        print(image)
        images.append(image)

# Process the images as required
# ...

# Example: Showing the first image
if images:
    images[0].show()
else:
    print("No images found in the directory.")


No images found in the directory.


## 파라미터 및 모델 설계

In [ ]:
batch_size = 32 #128
num_epochs = 40
image_size = (299, 299)
num_classes = 56

In [ ]:
# Load the InceptionV3 model
image_input = Input(shape=(*image_size, 3))
inception_model = InceptionV3(input_tensor=image_input, weights='imagenet', include_top=False) #top_layer는 우리 음식갯수에 맞게

In [ ]:
# Add a new classification layer
x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
class_outputs = Dense(num_classes, activation='softmax')(x)

In [ ]:
!pip install pillow scikit-learn

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

### 1안

In [ ]:
# 이미지의 각 픽셀을 255로 나눠주는 것은 필수!

train_datagen = ImageDataGenerator(
    rescale = 1./255,
)

# train_datagen = ImageDataGenerator(
#     rescale = 1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

val_datagen = ImageDataGenerator(rescale = 1./255) # 검증, 테스트 데이터는 augmentation 하면 안됨!
# test_datagen = ImageDataGenerator(rescale= 1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Inception_model_data/101_105',
    target_size = (299, 299),
    batch_size = 16, # gpu 연산 -> 2의 거듭제곱꼴로 맞추는 것이 좋다고 함
    seed = 42
)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Inception_model_data/101_105',
    target_size = (299, 299),
    batch_size = 16, # gpu 연산 -> 2의 거듭제곱꼴로 맞추는 것이 좋다고 함
    shuffle=True,
    seed = 42
)

# test_generator = test_datagen.flow_from_directory(
#     '/content/drive/MyDrive/Inception_model_data/101_105',
#     target_size = (299, 299),
#     batch_size = 16, # gpu 연산 -> 2의 거듭제곱꼴로 맞추는 것이 좋다고 함
#     seed = 42
# )

Found 55998 images belonging to 56 classes.
Found 55998 images belonging to 56 classes.


In [ ]:
#데이터 저장 용도
import numpy as np

# 데이터와 라벨을 저장할 리스트 초기화
data = []
labels = []

# 반복해서 모든 배치를 추출
for _ in range(len(test_generator)):
    batch_data, batch_labels = test_generator.next()
    data.append(batch_data)
    labels.append(batch_labels)

# Numpy 배열로 변환
data = np.vstack(data)
labels = np.hstack(labels)

# .npy 파일로 저장
np.save('/content/drive/MyDrive/Inception_model_data/saved_data_generator/test_data.npy', data)
np.save('/content/drive/MyDrive/Inception_model_data/saved_data_generator/test_labels.npy', labels)

print("Data and labels have been saved as .npy files.")


In [ ]:
# generator 출력 살펴보기
for data_batch, labels_batch in train_generator:
    print("배치 데이터 크기 : ", data_batch.shape)
    print("배치 레이블 크기 : ", labels_batch.shape)
    break

배치 데이터 크기 :  (16, 299, 299, 3)
배치 레이블 크기 :  (16, 56)


In [ ]:
K.clear_session()

with tf.device('/device:GPU:0'): # GPU 사용
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))

    for layer in base_model.layers:
      layer.trainable = False

    # Add new layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    class_outputs = Dense(num_classes, activation='softmax')(x)

    # x = AveragePooling2D(pool_size=(8, 8))(x)
    # x = Dropout(.4)(x)
    # x = Flatten()(x)
    # x = Dropout(.4)(x)
    # predictions = Dense(150,init='glorot_uniform',
    #                     W_regularizer=l2(.0005), activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=class_outputs)

    # Compile the model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)  # 조기 종료 콜백 함수 정의
    cb_checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Inception_model_data/saved_data_generator/checkpoint',
                                     monitor='val_accuracy', verbose=1, save_best_only=True)  # 체크포인트 저장

# 모델 요약 출력
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0][0

In [ ]:
# lr_scheduler : epoch 증가함에 따라 learning_rate를 작게 해 cosf_function이 최적값 주변에서 맴도는 것을 완화!

def schedule(epoch):
    if epoch < 15:
        return .01
    elif epoch < 28:
        return .002
    else:
        return .0004

lr_scheduler = LearningRateScheduler(schedule)

In [ ]:
history2 = model.fit(train_generator,
                     epochs=30,
                     steps_per_epoch=len(train_generator),
                     validation_data=val_generator,
                     validation_steps=len(val_generator),
                     use_multiprocessing=True,
                     callbacks=[cb_checkpoint, early_stopping, lr_scheduler])

Epoch 1/30


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


   5/3500 [..............................] - ETA: 12:11:14 - loss: 4.2294 - accuracy: 0.0000e+00

Process Keras_worker_ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/data_utils.py", line 648, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py", line 116, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py", line 370, in _get_batches_of_transformed_samples
    img = image_utils.load_img(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/image_utils.py", line 423, in load_img
    img = pil_i

### 2안

In [ ]:
# 2안
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import gc  # 가비지 컬렉션 모듈

# 데이터 디렉토리 설정
data_dir = '/content/drive/MyDrive/Inception_model_data/101_105'
target_size = (224, 224)
batch_size = 32  # 적절한 배치 크기로 설정

# 클래스 폴더를 5개씩 묶어서 처리
class_names = os.listdir(data_dir)
class_chunks = [class_names[i:i+5] for i in range(0, len(class_names), 5)]

# 데이터를 저장할 경로
output_dir = '/content/drive/MyDrive/Inception_model_data/saved_data'
os.makedirs(output_dir, exist_ok=True)

# 라벨 바이너리 인코더 초기화
lb = LabelBinarizer()

#가장 최근에 끊긴 train_labels_chunk로 lb.fit_transform 하기  > 0-3부터 lb fit_transform 다시하고 돌리기..
# loaded_label_array = np.load(os.path.join(output_dir, f'train_labels_chunk_6.npy'))
# label_array = lb.fit_transform(loaded_label_array)

# 반복문을 통해 각 묶음을 처리
for i, chunk in enumerate(class_chunks):

    # if i in [0, 1, 2, 3, 4, 5, 6]: # chunk 3까지 저장후 끊겨서
      # continue

    data_list = []
    label_list = []

    print(f"Processing chunk {i+1}/{len(class_chunks)}...")
    for class_name in chunk:
        class_dir = os.path.join(data_dir, class_name)
        print(class_name, " 시작")
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                img = Image.open(img_path).resize(target_size)
                img_array = np.array(img)
                img_array = preprocess_input(img_array)
                data_list.append(img_array)
                label_list.append(class_name)

    # 데이터를 Numpy 배열로 변환
    data_array = np.array(data_list)
    label_array = np.array(label_list)

    # 라벨을 원핫 인코딩
    if i == 0:
        label_array = lb.fit_transform(label_array)
    else:
        label_array = lb.transform(label_array)

    # 데이터 저장
    np.save(os.path.join(output_dir, f'train_data_chunk_{i}.npy'), data_array)
    np.save(os.path.join(output_dir, f'train_labels_chunk_{i}.npy'), label_array)

    print(f"Chunk {i+1} processed and saved.")

    # 메모리 해제
    del data_list
    del label_list
    del data_array
    del label_array
    gc.collect()  # 가비지 컬렉션 명시적 호출

# 데이터 합치기
print("Combining all chunks...")

train_data_list = []
train_labels_list = []

for i in range(len(class_chunks)):
    train_data_chunk = np.load(os.path.join(output_dir, f'train_data_chunk_{i}.npy'))
    train_labels_chunk = np.load(os.path.join(output_dir, f'train_labels_chunk_{i}.npy'))

    train_data_list.append(train_data_chunk)
    train_labels_list.append(train_labels_chunk)

# Numpy 배열로 합치기
combined_train_data = np.concatenate(train_data_list, axis=0)
combined_train_labels = np.concatenate(train_labels_list, axis=0)

# 학습 데이터와 검증 데이터로 분리
train_data, val_data, train_labels, val_labels = train_test_split(combined_train_data, combined_train_labels, test_size=0.2, random_state=42)

# 최종 데이터 저장
np.save(os.path.join(output_dir, 'train_data.npy'), train_data)
np.save(os.path.join(output_dir, 'train_labels.npy'), train_labels)
np.save(os.path.join(output_dir, 'val_data.npy'), val_data)
np.save(os.path.join(output_dir, 'val_labels.npy'), val_labels)

print("All data and labels have been saved successfully.")


Processing chunk 8/12...
김치볶음밥  시작
그라탕  시작
김  시작
껌  시작
까르보나라  시작
Chunk 8 processed and saved.
Processing chunk 9/12...
김치국  시작
꼬막  시작
꿀  시작
단호박  시작
누텔라  시작
Chunk 9 processed and saved.
Processing chunk 10/12...
단감  시작
녹색피망  시작
달걀볶음밥  시작
나시고랭  시작
다시마  시작
Chunk 10 processed and saved.
Processing chunk 11/12...
다크초콜릿  시작
냉면  시작
달걀  시작
달걀후라이  시작
달걀샌드위치  시작
Chunk 11 processed and saved.
Processing chunk 12/12...
달걀비빔밥  시작
Chunk 12 processed and saved.
Combining all chunks...


In [ ]:
# 2안
# Create the model
model = Model(inputs=inception_model.input, outputs=class_outputs) #입력 레이어 출력 레이어 설정

# Compile the model
# 손실 함수 : categorical_crossentropy
# 옵티마이저 : Adam , 학습률(lr) : 0.001
# 성능 평가 기준 :  accuracy
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Train the model and store the history
# 기본 배치 사이즈 : 32
history = model.fit(train_data, train_labels, epochs=num_epochs, batch_size=batch_size, validation_data=(val_data, val_labels))

# Save the model
model.save('food_classification_model_inceptionV3_v1.h5')